<a href="https://colab.research.google.com/github/Sayed-Ali-Raza-Naqvi/mlops-zoomcamp-homeworks/blob/main/Homework_Module1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [68]:
import pandas as pd
import pyarrow
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [43]:
df = pd.read_parquet('/content/yellow_tripdata_2023-01.parquet')

In [44]:
df.shape

(3066766, 19)

In [45]:
df.tpep_dropoff_datetime = pd.to_datetime(df.tpep_dropoff_datetime)
df.tpep_pickup_datetime = pd.to_datetime(df.tpep_pickup_datetime)

In [46]:
df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime

In [47]:
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,duration
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00,0 days 00:08:26
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00,0 days 00:06:19
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.9,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00,0 days 00:12:45
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.1,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25,0 days 00:09:37
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.4,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00,0 days 00:10:50


In [48]:
df.duration = df.duration.apply(lambda x: x.total_seconds()/60)

In [49]:
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,duration
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00,8.433333
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00,6.316667
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.9,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00,12.750000
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.1,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25,9.616667
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.4,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00,10.833333


In [50]:
df['duration'].std()

42.59435124195458

In [51]:
((df['duration'] >= 1) & (df['duration'] <= 60)).mean()

0.9812202822125979

In [52]:
data_for_encoding = ['PULocationID', 'DOLocationID']

In [53]:
df[data_for_encoding].dtypes

PULocationID    int64
DOLocationID    int64
dtype: object

In [54]:
df[data_for_encoding] = df[data_for_encoding].astype(str)

In [56]:
df.dtypes

VendorID                          int64
tpep_pickup_datetime     datetime64[us]
tpep_dropoff_datetime    datetime64[us]
passenger_count                 float64
trip_distance                   float64
RatecodeID                      float64
store_and_fwd_flag               object
PULocationID                     object
DOLocationID                     object
payment_type                      int64
fare_amount                     float64
extra                           float64
mta_tax                         float64
tip_amount                      float64
tolls_amount                    float64
improvement_surcharge           float64
total_amount                    float64
congestion_surcharge            float64
airport_fee                     float64
duration                        float64
dtype: object

In [58]:
dict_vectorizer = DictVectorizer()

In [60]:
train_dict = df[data_for_encoding].to_dict(orient='records')

In [61]:
X_train = dict_vectorizer.fit_transform(train_dict)

In [62]:
X_train.shape

(3066766, 518)

In [64]:
Y_train = df['duration'].values

In [66]:
clf = LinearRegression()

In [85]:
clf.fit(X_train, Y_train)

LinearRegression()

In [90]:
Y_pred = clf.predict(X_train)

In [73]:
mean_squared_error(Y_train, Y_pred, squared=False)

41.99649769068602

In [74]:
df_val = pd.read_parquet('/content/yellow_tripdata_2023-02.parquet')

In [75]:
df_val.tpep_dropoff_datetime = pd.to_datetime(df.tpep_dropoff_datetime)
df_val.tpep_pickup_datetime = pd.to_datetime(df.tpep_pickup_datetime)

In [76]:
df_val['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime

In [78]:
df_val.duration = df_val.duration.apply(lambda x: x.total_seconds()/60)

In [79]:
val_data_for_encoding = ['PULocationID', 'DOLocationID']

In [80]:
df_val[val_data_for_encoding] = df_val[val_data_for_encoding].astype(str)

In [81]:
dict_vectorizer2 = DictVectorizer()

In [82]:
val_train_dict = df_val[val_data_for_encoding].to_dict(orient='records')

In [83]:
X_val = dict_vectorizer2.fit_transform(val_train_dict)

In [87]:
Y_val = df_val['duration'].values

In [92]:
clf2 = LinearRegression()

In [98]:
clf2.fit(X_val, Y_val)

LinearRegression()

In [99]:
Y_val_pred = clf2.predict(X_val)

In [102]:
mean_squared_error(Y_val, Y_val_pred, squared=False)

43.211677888214375